<a href="https://colab.research.google.com/github/Shayee0778/Hamoye-Model-Maker/blob/main/Model_makers_Premiere_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drought Data Analysis By Team ModelMakers

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

Topic -

In [277]:
 #import the needed libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [278]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Load the dataset
df= pd.read_excel('/content/drive/MyDrive/public_emdat_custom_request.xlsx')
df.head()

# New section

In [ ]:
#check shape of dataset
df.shape

In [ ]:
#check for the data type of the columns
df.info()

In [ ]:
#check for the data description
df.describe()

In [ ]:
#check for null values
df.isnull().sum()

In [ ]:
#dropping all completely empty columns
df= df.dropna(axis='columns', how='all')
df.head()

In [ ]:
#checking null values
df.isnull().sum()

In [ ]:
#checking duplicate values
df.duplicated().sum()

In [ ]:
#removing other insignificant columns with large null values, columns with only one unique value and high missing values
cols_to_drop = ['DisNo.', 'Classification Key',  "Magnitude", "AID Contribution ('000 US$)","End Day", "Start Day","External IDs",
                 "AID Contribution ('000 US$)",'Start Day','Historic','Disaster Group',
                'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'Region', 'Magnitude Scale', 'Admin Units',	'Entry Date',	'Last Update', 'No. Affected']
df.drop(columns=cols_to_drop, axis=1, inplace=True)
df.columns

In [ ]:
#checking the null values
df.isnull().sum().sort_values()

In [ ]:
#checking the columns data type
df.info()

In [ ]:
#Rechecking empty rows
df.isnull().sum()

In [ ]:
#Fill empty rows with 0(zero) for the specified columns to indicate that the month number was not specified
columns_to_fill = ['Start Month', 'End Month']
for column_name in columns_to_fill:
    df.loc[df[column_name].isna(), column_name] = 0
df.head()

In [ ]:
#Rechecking empty rows
df.isnull().sum()

In [ ]:
#Fill empty rows with 'not specified' for the columns with object data type
columns_to_fill = ['Origin', 'Location','Associated Types']
for column_name in columns_to_fill:
    df.loc[df[column_name].isna(), column_name] = 'Not specified'
df.head()

In [ ]:
#Rechecking empty rows
df.isnull().sum()

In [ ]:

# Change data types
type_dict = { #"Total Affected" : "int64",
             "Country": 'category',
             "Subregion": "category",
             "Origin": "category",
             "OFDA Response": "category",
             "Appeal": "category",
             "Declaration": "category",
             'Start Year': 'category',
             'End Year': 'category'
}

for key, value in type_dict.items():
  df[key] = df[key].astype(value)
df.info()

In [296]:
#"Total Affected" : "int64"

df['Total Affected'] = pd.to_numeric(df['Total Affected'], errors='coerce').fillna(0).astype(int)

In [ ]:
# Import the UNICEF regional data
regions_df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/JME_Regional-Classifications.xlsx")[['ISO Code', 'UN Sub Region', 'World Bank Income Groups Combined']]
regions_df.head()

In [ ]:
# merge the regions data with the original table
df = pd.merge(left=df, right=regions_df, left_on = 'ISO', right_on = 'ISO Code', how='left')
df = df.drop(columns=['ISO Code'])
df

In [299]:
# Rename column
df = df.rename(columns={
    "World Bank Income Groups Combined": "Income group"
})
# Export cleaned data
#df.to_csv('drought_data_cleaned.csv', index=False)

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

## Research Questions

the dataset contains information on the occurence of drought in africa. This analysis aims to answer the following:

### 1. who were the worst hit? what is their distribution per:

- country
- region
- income group

### 2. what are the effects of drought?
- direct
- secondary

In [ ]:
df.head(1)

In [ ]:
map_df = df.groupby('Country').sum().reset_index()
map_df
# ['Total Affected'].describe()

**UNIVARIATE**

In [ ]:

# px.boxplot(data_frame=map_df2, column'Total Affected')
px.box(data_frame=map_df, x='Total Affected')
# px.box(map_df, y='ISO', x='Total Affected', range_x = [10000000, 30000000])

__OBSERVATION__

From the above analysis, the following countries were the most hit in terms of the number of persons affected by drought:
- Ethiopia
- Kenya
- Somalia
- Zimbabwe
- Nigeria.

Interestingly, __top 3 of the top 5 countries most Affected were still eastern African countries.__ Why exactly is Eastern Africa most affected by drought?
Could it be a geographical factor or as a result of poor emergency response on the part of the government?

This question calls for further investigations as our data cannot provide an answer to it!


In [ ]:
plt.figure(figsize=(5, 6))
sns.countplot(data=df, x="OFDA Response")
plt.title('Response Frequency of the United States')
plt.xlabel("OFDA Response")
plt.show()

In [ ]:
#countplot to show the Frequecy of disaster
df['Associated Types'].value_counts().sort_values().plot(kind='barh')
plt.title('Disaster distribution plot')
plt.xlabel('Frequency')
plt.ylabel('Disaster')
plt.show()
#It can be observed that the most occuring disaster is Food Shortage, followed by Famine;  It cane be observed that most of drought that happended in Nigeria was as a
#result of food shortage or Famine


In [ ]:
plt.figure(figsize=(10,8))
df['Country'].value_counts().sort_values().plot(kind='barh')
plt.title('Country and Number of Drought occurence')
plt.xlabel('Frequency')
plt.ylabel('Country')
plt.show()

#Somalia and Mozambique have expereiecnce more drought than any other country in Africa since 2000

In [ ]:
df[df['Start Year'] == 2022]

In [ ]:
df['Start Year'].value_counts().sort_values(ascending=True).plot(kind='barh')
plt.title('Year their respective number of occurence')
plt.xlabel('Frequency')
plt.ylabel('Year')
plt.show()
#2022 recorded the hihest occurence of drought, followed by 2005 and 2013 being the year with the least recor of drought

In [ ]:
#pie chart for disaster group
x = df['Subregion'].value_counts()
plt.pie(x.values,
		labels=x.index,
		autopct='%1.1f%%')
plt.title('Disaster distribution by Subregion')
plt.show()
#97.6% of drought that happened in Africa has been from Sub-Saharan Africa and Northern Africa having 2.4%

In [ ]:

month_dict = {1.0:'Jan', 2.0:'Feb',3.0:'Mar', 4.:'Apr', 5. : 'May', 6.:'Jun', 7.:'Jul', 8. :'Aug',  9.:'Sep', 10. :'Oct', 11. :'Nov', 12.: 'Dec', 0.0:'Not Specified'}
df['Start Month'].replace(month_dict, inplace = True)
df['Start Month'].value_counts().sort_values(ascending=False).plot(kind='bar')
plt.title('Month and their respective number of occurence')
plt.xlabel('Month')
plt.ylabel('Frequency')
plt.show()
#Drough occur mostly in the month of Jan (dry season)

**BIVARIATE**

In [ ]:
# How many persons were affected in each subregion for the period under review?
count_by_subregion = df.groupby("UN Sub Region").sum().reset_index()
count_by_subregion

In [ ]:
# compute percentages for each subregion
count_by_subregion['Percentage'] = (count_by_subregion['Total Affected'] / sum(count_by_subregion['Total Affected'])) * 100
count_by_subregion

In [ ]:
px.pie(data_frame=count_by_subregion, values='Total Affected', names = "UN Sub Region", title="Number of persons affected per UN Subregion")

__OBSERVATION__:
The visual above shows that __54.9%__ of person affected come from eastern africa. This means that one in every two persons affected by drought in africa is from eastern africa.

In [ ]:
count_by_country = df.groupby("Country").sum().sort_values(by='Total Affected').reset_index()
count_by_country

In [ ]:
# Create a temporary dataframe containing some columns of our main dataframe
# so that we can join them to the aggregated dataframe for the map

temp_df = df[['Country', 'UN Sub Region', 'Income group', 'ISO']]
temp_df = temp_df.drop_duplicates()
temp_df

In [ ]:
# Now, we can join our temp_df with the map_df dataframe using the "Country" common column:
map_df = pd.merge(left=map_df, right=temp_df, on="Country", how='left')
map_df

Now, we have the aggregated table containing other values that we can use for our map.
Next, let's group the "Total Affected" column into intervals reflecting the severity of occurence:

In [ ]:
# Split the "Total Affected" column into intervals
bin_edges = [0, 5000000, 15000000, 30000000, map_df["Total Affected"].max()]
bin_labels = ["Low Severity (0-5M)", "Moderate Severity (>5M - 15M)", "High Severity (>15M - 30M)", "Critical Severity (>30M)"]
map_df["Severity Level"] = pd.cut(x=map_df["Total Affected"], bins=bin_edges, labels=bin_labels, right=True)
map_df.head(1)

In [ ]:
color_discrete_map = {
    "Critical Severity (>30M)": "#A70100",
    "High Severity (>15M - 30M)": "#D93F00",
    "Moderate Severity (>5M - 15M)": "#FD8E2A",
    "Low Severity (0-5M)": "#FFD983"
}

map_plot = px.choropleth(data_frame=map_df, locations="ISO", locationmode="ISO-3", scope='africa',
              color='Severity Level', color_discrete_map=color_discrete_map,
             hover_data=map_df[["Severity Level", "Total Affected", "Income group", "UN Sub Region"]],
             height=600, width=800)

# update layout
map_plot.update_layout(title="Drought Severity Level by Country",
                      margin={"r":0, "t":40, "l":0, "b":0})
map_plot.show()

In [ ]:
count_by_income = df.groupby("Income group").sum().reset_index()
count_by_income

In [ ]:
count_by_income['Percentage'] = (count_by_income['Total Affected'] / sum(count_by_income['Total Affected'])) * 100
count_by_income

In [ ]:
px.bar(data_frame = count_by_income, x = "Income group", y = "Total Affected", text="Percentage")

__OBSERVATION__
The bar chart shows that countries with low income were affected more than those in middle income by more than two times! It therefore gives a pointer that the income level of a country most definitely affects the number of persons involved in drought

Line chart showing the number of people affected by forced migration over time

In [321]:
#Group the data by time
df_grouped = df.groupby(['Start Year', 'Total Affected'])

In [ ]:
#Calculate the mean number of people affected by forced migration for each time period
df_mean = df_grouped.mean()

In [ ]:
df_mean

In [324]:
#Reset the index
df_mean = df.reset_index()

In [325]:
#Sort the data by time
df_mean = df.sort_values(by='Start Year')

In [ ]:
df_mean = df_mean.groupby("Start Year").mean().reset_index()

In [ ]:
df_mean

In [ ]:
#Create the line chart
plt.figure(figsize=(10, 6))
plt.plot(df_mean['Start Year'], df_mean['Total Affected'])
plt.title('Number of People Affected  over Time')
plt.xlabel('Start Year')
plt.ylabel('Total Affected')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_mean['Start Year'], df_mean['CPI'])
plt.title('CPI over Time')
plt.xlabel('Start Year')
plt.ylabel('Consumer Price Index')
plt.show()

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x="UN Sub Region", hue="OFDA Response")
plt.title('Response Frequency of the United States per region')
plt.xlabel('African Region')
plt.show()

In [ ]:
df.head(1)

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df[df['Start Month'] != 'Not Specified'], hue="UN Sub Region", x="Start Month")
plt.title('Monthly Occurence of Drough per region')
plt.xlabel('Month')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df["Appeal"].value_counts()

In [ ]:
df["OFDA Response"].value_counts()

In [ ]:
#distribution of death for each year
plt.figure(figsize=(15,5))
#df_copy_ana[['Start Year','Total Deaths']].set_index('Start Year').plot()
sns.lineplot(data=df, x='Start Year', y='Total Deaths', errorbar=None)
plt.title('Total Death with number of Yearly occurence')
plt.xlabel('Year')
plt.ylabel('Total Death')
plt.show()

#The total number of death recorded has reduced after a peak in 2010, we really cannot say since this columns has alot of missing values

In [ ]:
#scatter plot for Total Affected and Total Death
plt.figure(figsize=(15,8))
sns.scatterplot(data=df, x='Total Affected', y="Total Deaths", color='steelblue')
plt.title('Total Affected vs Total Death')
plt.xlabel('Total Affected')
plt.ylabel('Total Deaths')
plt.show()

In [ ]:
#Country anf frequency of Associated Type of disaster
_ = pd.crosstab( df['Country'],df['Associated Types'])
_.plot(kind="barh", stacked=True, rot=0, figsize=(8, 8) )
plt.title('Stacked bar chart of Country and the Associated type of Drought ')
plt.xlabel('Frequency')
plt.ylabel('Country')
plt.show()


In [ ]:
#barchart to show the Total Damages spent on disaster
country_with_high_death = df.groupby('Country')[['Country', "Total Deaths"]].sum()\
.sort_values(by="Total Deaths", ascending=False).reset_index().head(10)
country_with_high_death

In [ ]:
plt.figure(figsize=(15,5))
#df[['Start Year','Total Deaths']].set_index('Start Year').plot()
sns.lineplot(data=df[df['Country'] == 'Ethiopia'], x='Start Year', y='Total Affected', errorbar=None,color='green' )
sns.lineplot(data=df[df['Country'] == 'South Africa'], x='Start Year', y='Total Affected', errorbar=None,color='blue')
sns.lineplot(data=df[df['Country'] == 'Kenya'], x='Start Year', y='Total Affected', errorbar=None, color='red')
sns.lineplot(data=df[df['Country'] == 'Niger'], x='Start Year', y='Total Affected', errorbar=None, color='purple')
sns.lineplot(data=df[df['Country'] == 'Somalia'], x='Start Year', y='Total Affected', errorbar=None, color='orange')
plt.legend(labels=['Ethiopia', 'South Africa', 'Kenya', 'Niger', 'Somalia'])
plt.title('Yearly distribution plot')
plt.xlabel('Year')
plt.ylabel('Total AFfected')
plt.show()

